<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [1]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-fes03jkg
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-fes03jkg
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit a1be39c0b7c0cf693ac4cafb15f1ac5e73c2d033
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.6 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42026 sha256=6f72429478e2fa60e6ff794fcd4b8667ce8c2d0b73c95e32fa8e961f67339ad6
  Stored in directory: /tmp/pip-ephem-wheel-cache-r4k0ikwh/wheels/3e/1c/74/97ec620813c1bb106679625869f7409ca5be8735d0a74457bf
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

### Setup and Configuration

In [41]:
# @title Enable DataFrame Formaatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [42]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [43]:
# @title Initialize Widgets
import ipywidgets as widgets
output = widgets.Output()

In [44]:
# @title Define Common Functions

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def rank(code, period='2y', tickers_getter=get_tickers,
         ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)

    output.clear_output()
    with output:
        rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                               ref_ticker=ref_ticker)

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Save to CSV
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust


def increasing_rs_filter(df):
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        #& (df["Percentile"] > 90)
    ]

### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### Error Messages

In [45]:
display(output)

Output()

### RS Rating and Ranking

In [47]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
display(rank_stock)
display(rank_indust)



***
Your "SPX+DJIA+NDX+SOX_stocks_2y_20240809.csv" is in the "out" folder.
Your "SPX+DJIA+NDX+SOX_industries_2y_20240809.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
462,VST,Utilities,Utilities - Independent Power Producers,156.88,192.00,238.58,140.22,99,99,99,97,1
264,NVDA,Technology,Semiconductors,153.47,187.10,169.40,177.27,99,99,99,99,2
435,IRM,Real Estate,REIT - Specialty,145.72,124.08,116.53,103.70,99,95,88,69,3
508,FICO,Technology,Software - Application,143.97,131.59,119.32,131.11,99,97,91,96,4
266,GDDY,Technology,Software - Infrastructure,141.55,127.27,132.61,118.52,99,97,97,92,5
...,...,...,...,...,...,...,...,...,...,...,...,...
510,ALB,Basic Materials,Specialty Chemicals,56.02,55.55,87.38,63.10,0,0,19,0,517
212,DXCM,Healthcare,Medical Devices,56.01,80.24,97.70,92.39,0,15,48,40,518
36,INTC,Technology,Semiconductors,54.40,77.54,73.35,113.24,0,10,3,86,519
375,WBA,Healthcare,Pharmaceutical Retailers,48.64,46.14,68.46,74.30,0,0,0,4,520


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
84,Utilities - Independent Power Producers,Utilities,146.16,163.82,205.05,128.39,"VST,NRG",99,99,99,98,1
34,Medical Care Facilities,Healthcare,120.70,101.34,109.72,107.30,"UHS,HCA,DVA",98,89,92,83,2
75,Oil & Gas Midstream,Energy,117.18,108.45,111.29,97.00,"TRGP,OKE,KMI,WMB",97,96,95,49,3
78,Conglomerates,Industrials,114.84,99.05,102.60,85.44,"MMM,HON",96,75,67,12,4
56,REIT - Healthcare Facilities,Real Estate,114.56,99.87,99.82,87.93,"VTR,WELL,DOC",95,82,61,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...
81,Oil & Gas Equipment & Services,Energy,84.67,79.17,91.61,81.75,"BKR,SLB,HAL",4,5,18,5,87
12,Beverages - Brewers,Consumer Defensive,83.57,77.74,87.04,90.99,"STZ,TAP",3,3,5,26,88
32,Auto Parts,Consumer Cyclical,79.66,75.35,87.89,77.96,"GPC,BWA,LKQ,APTV",2,0,6,2,89
79,Airlines,Industrials,77.38,78.44,102.88,91.48,"DAL,LUV,UAL,AAL",1,4,69,27,90


In [48]:
# @title Print Top Percentile Stocks
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)


number of filtered tickers: 52
['VST', 'NVDA', 'IRM', 'FICO', 'GDDY', 'AXON', 'HWM', 'NRG', 'MMM', 'TRGP', 'KKR', 'UHS', 'TSM', 'GE', 'MPWR', 'ANET', 'MHK', 'ISRG', 'AVGO', 'NTAP', 'RCL', 'TYL', 'K', 'VTR', 'EFX', 'CBRE', 'CEG', 'BRO', 'META', 'CFG', 'WELL', 'GRMN', 'HCA', 'COST', 'MSI', 'GS', 'EXR', 'LLY', 'COHR', 'CTAS', 'DHI', 'KLAC', 'GEN', 'TMUS', 'PGR', 'HIG', 'TT', 'IP', 'FITB', 'EBAY', 'ICE', 'DVA']


In [49]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = increasing_rs_filter(rank_stock)
display(filtered_rank_stock)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
435,IRM,Real Estate,REIT - Specialty,145.72,124.08,116.53,103.70,99,95,88,69,3
253,TRGP,Energy,Oil & Gas Midstream,132.60,125.76,122.31,106.44,98,96,93,75,10
99,TYL,Technology,Software - Application,121.06,108.40,102.57,99.21,95,87,65,57,22
108,VTR,Real Estate,REIT - Healthcare Facilities,118.93,101.50,96.67,85.34,95,74,46,20,24
402,WELL,Real Estate,REIT - Healthcare Facilities,116.95,103.73,103.56,100.23,94,78,68,60,31
394,MSI,Technology,Communication Equipment,116.60,105.61,103.73,95.30,93,82,68,48,35
429,EXR,Real Estate,REIT - Industrial,116.35,99.11,97.98,90.57,93,68,49,36,37
393,IP,Consumer Cyclical,Packaging & Containers,114.94,106.32,105.74,91.51,90,83,73,38,48
409,PM,Consumer Defensive,Tobacco,114.20,99.61,97.58,84.85,89,69,48,19,53
215,AFL,Financial Services,Insurance - Life,113.96,102.40,100.54,93.68,89,75,59,44,55


### RS Rating and Ranking for Taiwan Stocks

In [50]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')
display(tw_stocks)
display(tw_industries)



***
Your "TWSE+TPEX_stocks_2y_20240809.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_2y_20240809.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1076,6144.TWO,Communication Services,Entertainment,453.14,257.19,116.92,83.50,99.0,99,87,17,1
1344,1799.TWO,Healthcare,Drug Manufacturers - Specialty & Generic,442.29,330.83,208.06,208.31,99.0,99,99,98,2
24,8374.TW,Industrials,Industrial Distribution,382.67,261.31,83.62,83.34,99.0,99,40,17,3
1779,2365.TW,Technology,Computer Hardware,356.52,284.11,139.38,173.13,99.0,99,94,97,4
1816,4562.TW,Industrials,Specialty Industrial Machinery,352.39,302.62,159.95,83.68,99.0,99,96,18,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1559,6108.TW,Technology,Electronic Components,NaN,74.79,85.71,119.52,NaN,20,45,85,2004
1576,2073.TWO,Basic Materials,Steel,NaN,76.25,79.66,86.51,NaN,25,28,26,2005
1699,5016.TWO,Basic Materials,Steel,NaN,76.60,79.65,85.77,NaN,26,27,23,2006
1842,00643K.TW,Unknown,Unknown,NaN,76.46,80.19,87.10,NaN,25,29,28,2007


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
18,Industrial Distribution,Industrials,162.52,132.87,85.73,90.82,"8374.TW,911608.TW,3114.TWO,2373.TW",99.0,99,37,36,1
14,Real Estate - Development,Real Estate,137.74,122.84,128.96,107.91,"2524.TW,5508.TWO,3188.TWO,5455.TWO,2537.TW,143...",98.0,96,99,87,2
40,Entertainment,Communication Services,133.96,103.73,95.48,97.20,"6144.TWO,4806.TWO,6596.TWO,8446.TWO,6184.TW,84...",97.0,89,74,68,3
86,Real Estate - Diversified,Real Estate,127.65,120.26,126.89,117.46,"1438.TW,9946.TW,6219.TWO,2520.TW,2545.TW,5512....",96.0,95,98,97,4
71,Utilities - Renewable,Utilities,121.18,129.49,103.07,108.70,"6869.TW,6873.TW,6806.TW,8087.TWO",95.0,97,89,90,5
...,...,...,...,...,...,...,...,...,...,...,...,...
34,Steel,Basic Materials,NaN,78.95,86.40,91.46,"2025.TW,2008.TW,2015.TW,2006.TW,5014.TWO,2031....",NaN,19,42,41,92
37,Electrical Equipment & Parts,Industrials,NaN,96.94,94.15,98.39,"2438.TW,6788.TWO,2061.TWO,1616.TW,1615.TW,1503...",NaN,75,69,70,93
52,Credit Services,Financial Services,NaN,72.78,73.34,86.30,"6592A.TW,9941.TW,6592.TW,5871.TW,9941A.TW",NaN,5,4,13,94
61,Electronic Gaming & Multimedia,Communication Services,NaN,102.57,102.97,103.98,"3293.TWO,4946.TWO,3064.TWO,3086.TWO,4994.TW,75...",NaN,86,88,77,95


In [51]:
# @title Print Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)


number of filtered tickers: 100
['得利影', '易威', '羅昇', '昆盈', '穎漢', '均華', '光聖', '晶彩科', '皇昌', '新復興', '所羅門', '海悅', '福裕', '康全電訊', '欣巴巴', '福大', '慧友', '京城', '弘憶股', '新門', '彬台', '世紀', '全譜', '翔耀', '福懋油', '天品', '花王', '東捷', '永信建', '高鋒', '志聖', '慶騰', '弘塑', '訊舟', '均豪', '鑫科', '京晨科', '安國', '擎亞', '系微', '喬福', '三地開發', '天方能源', '及成', '鑫龍騰', '太普高', '天揚', '昇陽半導體', '昇益', '順藥', '聯上發', '和椿', '華友聯', '藥華藥', '旺矽', '鈊象', '泰偉', '德晉', '訊聯基因', '聯鈞', '泰金-KY', '藝舍-KY', '鏵友益', '錦明', '光明', '盟立', '晶悅', '錸德', '華城', '鈺邦', '坤悅', '宏碩系統', '益登', '雲豹能源', '惠特', '天剛', '富宇', '華義', '波力-KY', '森寶', '愛山林', '勝昱', '合騏', '直得', '大城地產', '信立', '佳能', '理銘', '大飲', '精湛', '達能', '泰金寶-DR', '迎廣', '晟銘電', '友威科', '精材', '德律', '信紘科', '宏旭-KY', '加捷生醫']


In [53]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

def stock_ticker_to_name(df_stocks):
    # Iterate over the 'Ticker' column in the tw_stocks DataFrame
    for index, row in df_stocks.iterrows():
        ticker = row['Ticker']
        stock_name = tw.stock_name(ticker)
        # Update the stock name back to the DataFrame
        df_stocks.at[index, 'Ticker'] = stock_name

def industry_tickers_to_names(df_industries):
    # Iterate over the 'Tickers' column in the tw_industries DataFrame
    for index, row in df_industries.iterrows():
        tickers = row['Tickers'].split(',')  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df_industries.at[index, 'Tickers'] = ','.join(stock_names)  # Join the names back into a comma-separated string

filtered_tw_stocks = increasing_rs_filter(tw_stocks)
stock_ticker_to_name(filtered_tw_stocks)
display(filtered_tw_stocks)

filtered_tw_industries = increasing_rs_filter(tw_industries)
industry_tickers_to_names(filtered_tw_industries)
display(filtered_tw_industries)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1076,得利影,Communication Services,Entertainment,453.14,257.19,116.92,83.50,99.0,99,87,17,1
24,羅昇,Industrials,Industrial Distribution,382.67,261.31,83.62,83.34,99.0,99,40,17,3
1816,穎漢,Industrials,Specialty Industrial Machinery,352.39,302.62,159.95,83.68,99.0,99,96,18,5
1504,均華,Technology,Semiconductors,331.40,309.49,236.10,167.45,99.0,99,99,96,6
274,新復興,Technology,Electronic Components,292.21,247.98,247.04,224.91,99.0,99,99,99,10
229,海悅,Real Estate,Real Estate Services,282.28,220.09,215.74,126.16,99.0,98,99,88,12
372,欣巴巴,Industrials,Engineering & Construction,264.48,157.31,123.81,115.12,99.0,96,90,82,15
375,京城,Real Estate,Real Estate - Development,246.44,151.31,138.69,87.36,99.0,96,94,29,18
1895,福懋油,Consumer Defensive,Packaged Foods,226.07,181.58,101.96,89.17,98.0,97,74,35,25
1647,東捷,Technology,Semiconductor Equipment & Materials,216.15,216.00,123.68,117.91,98.0,98,90,84,28


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
9,Specialty Industrial Machinery,Industrials,120.47,110.73,105.44,104.77,"穎漢,福裕,彬台,高鋒,志聖,喬福,太普高,和椿,盟立,惠特,精湛,友威科,信紘科,萬潤,台...",93.0,93,92,83,6
59,Specialty Business Services,Industrials,106.81,105.40,99.79,94.76,"花王,沈氏,白紗科,秋雨,關貿,政伸,信實,耕興,遠雄港",87.0,90,81,58,11
